## Introduction

This notebook combines selected macro features with apartment features and builds a pipeline to select the best model

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import label_binarize
from datetime import datetime as dt
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
datadir = '~/Desktop/sberbank_samen/data/'

In [3]:
X_train_clean = pd.read_csv(datadir + 'AptFeatures_train.csv')
X_test_clean = pd.read_csv(datadir + 'AptFeatures_test.csv')

df_macro = pd.read_csv(datadir + 'macro.csv')
macro_cols = pd.read_csv(datadir + 'macro_features_rf.csv', header=None)

In [4]:
y_train = pd.read_csv(datadir +'target_train.csv', header=None)

In [5]:
def extract_date_variables(input_data, date_in_index=False):
    
    '''This function extracts data variables from a series or index of a series. Returns
        a DataFrame with extracted variables'''

    if date_in_index:
        
        
        input_data['day'] = map(lambda x: str(x), input_data.index.day)
        input_data['year'] = map(lambda x: str(x), input_data.index.year)
        input_data['month'] = map(lambda x: str(x), input_data.index.month)

        #input_data['week_year'] = input_data['weeknr'] + '_' + input_data['year']
        #input_data['day_month_year'] = input_data['day'] + '_' + input_data['month'] + '_' + input_data['year']

        return input_data
    
    else:
        output_data = input_data.to_frame(name='timestamp')
        
        output_data['day'] = map(lambda x: str(x.day), output_data['timestamp'])
        output_data['year'] = map(lambda x: str(x.year), output_data['timestamp'])
        output_data['month'] = map(lambda x: str(x.month), output_data['timestamp'])

        #output_data['week_year']  = output_data['weeknr'] + '_' + output_data['year']
        #output_data['day_month_year'] = output_data['day'] + '_' + output_data['month'] + '_' + output_data['year']

        return output_data[['day', 'month', 'year']]

def check_columns(A, B):
    return dict(left_only = set(A.columns) - set(B.columns), right_only = set(B.columns)-set(A.columns))

## Preprocess + Combine data

Assign time variables

In [6]:
df_macro['timestamp'] = pd.to_datetime(df_macro['timestamp'])
X_train_clean['timestamp'] = pd.to_datetime(X_train_clean['timestamp'])
X_test_clean['timestamp'] = pd.to_datetime(X_test_clean['timestamp'])


X_train_clean = X_train_clean.drop(['month', 'month_year'], axis=1)
X_test_clean = X_test_clean.drop(['month', 'month_year'], axis=1)

In [7]:
df_macro = pd.concat([df_macro, extract_date_variables(df_macro['timestamp']) ], axis=1)
X_train_clean = pd.concat([X_train_clean, extract_date_variables(X_train_clean['timestamp']) ], axis=1)
X_test_clean = pd.concat([X_test_clean, extract_date_variables(X_test_clean['timestamp'])], axis=1)





In [8]:
X_train_clean.head()

,Unnamed: 0,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,...,office_sqm_3000,trc_count_3000,trc_sqm_3000,cafe_count_3000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,build_type_maxfloor,first_floor,last_floor,apt_count,day,month,year
0,0,2011-08-20,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,6.407578e+06,155572.0,0.189727,0.000070,9576.0,5001.0,5.0,10309.0,11065.0,5.0,0.0,240.0,1.0,0.0,7.0,3.0,no,0.0,16.0,1.0,no,no,no,no,no,no,no,no,86206.0,40477.0,45729.0,21154.0,11007.0,10147.0,98207.0,52277.0,45930.0,36211.0,...,251554.0,23.0,1419204.0,68.0,639.68,1079.37,859.52,5.0,21.0,22.0,16.0,3.0,1.0,0.0,2.0,4.0,0.0,0.0,21.0,1.0,13.09,13.31,29.0,807385.0,52.0,4036616.0,152.0,708.57,1185.71,947.14,12.0,39.0,48.0,40.0,9.0,4.0,0.0,13.0,22.0,1.0,0.0,52.0,4.0,NaN,False,False,3,20,8,2011
1,1,2011-08-23,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,9.589337e+06,115352.0,0.372602,0.049637,6880.0,3119.0,5.0,7759.0,6237.0,8.0,0.0,229.0,1.0,0.0,6.0,1.0,yes,1.0,3.0,0.0,no,no,no,no,no,no,no,no,76284.0,34200.0,42084.0,15727.0,7925.0,7802.0,70194.0,35622.0,34572.0,29431.0,...,821986.0,14.0,491565.0,30.0,631.03,1086.21,858.62,1.0,11.0,11.0,4.0,2.0,1.0,0.0,1.0,7.0,0.0,6.0,19.0,1.0,10.26,27.47,66.0,2690465.0,40.0,2034942.0,177.0,673.81,1148.81,911.31,9.0,49.0,65.0,36.0,15.0,3.0,0.0,15.0,29.0,1.0,10.0,66.0,14.0,NaN,False,False,3,23,8,2011
2,2,2011-08-27,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,4.808270e+06,101708.0,0.112560,0.118537,5879.0,1463.0,4.0,6207.0,5580.0,7.0,0.0,1183.0,1.0,0.0,5.0,1.0,no,0.0,0.0,1.0,no,no,no,yes,no,no,no,no,101982.0,46076.0,55906.0,13028.0,6835.0,6193.0,63388.0,31813.0,31575.0,25292.0,...,110856.0,7.0,52550.0,41.0,697.44,1192.31,944.87,2.0,9.0,17.0,9.0,3.0,1.0,0.0,0.0,11.0,0.0,0.0,20.0,6.0,13.69,21.58,43.0,1478160.0,35.0,1572990.0,122.0,702.68,1196.43,949.55,10.0,29.0,45.0,25.0,10.0,3.0,0.0,11.0,27.0,0.0,4.0,67.0,10.0,NaN,False,False,3,27,8,2011
3,3,2011-09-01,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,1.258354e+07,178473.0,0.194703,0.069753,13087.0,6839.0,9.0,13670.0,17063.0,10.0,0.0,NaN,1.0,0.0,17.0,6.0,no,0.0,11.0,4.0,no,no,no,no,no,no,no,no,21155.0,9828.0,11327.0,28563.0,14680.0,13883.0,120381.0,60040.0,60341.0,29529.0,...,167000.0,12.0,205756.0,32.0,718.75,1218.75,968.75,0.0,5.0,14.0,10.0,3.0,0.0,0.0,1.0,2.0,0.0,0.0,18.0,3.0,14.18,3.89,8.0,244166.0,22.0,942180.0,61.0,931.58,1552.63,1242.11,4.0,7.0,21.0,15.0,11.0,2.0,1.0,4.0,4.0,0.0,0.0,26.0,3.0,NaN,False,False,39,1,9,2011
4,4,20

Select relevant macro features

In [9]:
useful_cols = list(macro_cols.iloc[:,1].values)
useful_cols.extend(['day', 'month', 'year'])


df_macro_useful = df_macro[useful_cols]

# fill NA
df_macro_useful.fillna(method='ffill', inplace=True)
df_macro_useful.fillna(method='bfill', inplace=True)

/home/roman/anaconda2/envs/python_cursus/lib/python2.7/site-packages/pandas/core/frame.py:2842: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


Merge macro features with training set and test set

In [10]:
X_train_clean = X_train_clean.merge(df_macro_useful, on=['day', 'month', 'year'])
X_test_clean = X_test_clean.merge(df_macro_useful, on=['day', 'month', 'year'])


In [11]:
X_train_clean = X_train_clean.drop(['day', 'year', 'timestamp'], axis=1).iloc[:,1:]
X_test_clean  = X_test_clean.drop(['day', 'year', 'timestamp'], axis=1).iloc[:,1:]

In [12]:
X_train_clean.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,...,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,build_type_maxfloor,first_floor,last_floor,apt_count,month,balance_trade,micex_rgbi_tr,unprofitable_enterpr_share,ppi,rent_price_1room_eco,construction_value,mortgage_growth,rent_price_3room_bus,hot_water_share,rent_price_1room_bus,rent_price_2room_eco,deposits_growth
0,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,6.407578e+06,155572.0,0.189727,0.000070,9576.0,5001.0,5.0,10309.0,11065.0,5.0,0.0,240.0,1.0,0.0,7.0,3.0,no,0.0,16.0,1.0,no,no,no,no,no,no,no,no,86206.0,40477.0,45729.0,21154.0,11007.0,10147.0,98207.0,52277.0,45930.0,36211.0,10580.0,25631.0,...,16.0,3.0,1.0,0.0,2.0,4.0,0.0,0.0,21.0,1.0,13.09,13.31,29.0,807385.0,52.0,4036616.0,152.0,708.57,1185.71,947.14,12.0,39.0,48.0,40.0,9.0,4.0,0.0,13.0,22.0,1.0,0.0,52.0,4.0,NaN,False,False,3,8,15.459,131.16,0.292,420.7,29.07,549075.8,1.051914,77.93,95.7,47.85,36.77,0.009740
1,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,9.589337e+06,115352.0,0.372602,0.049637,6880.0,3119.0,5.0,7759.0,6237.0,8.0,0.0,229.0,1.0,0.0,6.0,1.0,yes,1.0,3.0,0.0,no,no,no,no,no,no,no,no,76284.0,34200.0,42084.0,15727.0,7925.0,7802.0,70194.0,35622.0,34572.0,29431.0,9266.0,20165.0,...,4.0,2.0,1.0,0.0,1.0,7.0,0.0,6.0,19.0,1.0,10.26,27.47,66.0,2690465.0,40.0,2034942.0,177.0,673.81,1148.81,911.31,9.0,49.0,65.0,36.0,15.0,3.0,0.0,15.0,29.0,1.0,10.0,66.0,14.0,NaN,False,False,3,8,15.459,131.45,0.292,420.7,29.07,549075.8,1.051914,77.93,95.7,47.85,36.77,0.009740
2,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,4.808270e+06,101708.0,0.112560,0.118537,5879.0,1463.0,4.0,6207.0,5580.0,7.0,0.0,1183.0,1.0,0.0,5.0,1.0,no,0.0,0.0,1.0,no,no,no,yes,no,no,no,no,101982.0,46076.0,55906.0,13028.0,6835.0,6193.0,63388.0,31813.0,31575.0,25292.0,7609.0,17683.0,...,9.0,3.0,1.0,0.0,0.0,11.0,0.0,0.0,20.0,6.0,13.69,21.58,43.0,1478160.0,35.0,1572990.0,122.0,702.68,1196.43,949.55,10.0,29.0,45.0,25.0,10.0,3.0,0.0,11.0,27.0,0.0,4.0,67.0,10.0,NaN,False,False,3,8,15.459,131.08,0.292,420.7,29.07,549075.8,1.051914,77.93,95.7,47.85,36.77,0.009740
3,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,1.258354e+07,178473.0,0.194703,0.069753,13087.0,6839.0,9.0,13670.0,17063.0,10.0,0.0,NaN,1.0,0.0,17.0,6.0,no,0.0,11.0,4.0,no,no,no,no,no,no,no,no,21155.0,9828.0,11327.0,28563.0,14680.0,13883.0,120381.0,60040.0,60341.0,29529.0,9083.0,20446.0,...,10.0,3.0,0.0,0.0,1.0,2.0,0.0,0.0,18.0,3.0,14.18,3.89,8.0,244166.0,22.0,942180.0,61.0,931.58,1552.63,1242.11,4.0,7.0,21.0,15.0,11.0,2.0,1.0,4.0,4.0,0.0,0.0,26.0,3.0,NaN,False,False,39,9,15.386,131.45,0.292,434.4,30.63,549075.8,1.04

## Dtypes

In [12]:
summary_X_train = pd.read_csv('summary_X_train.csv')

In [13]:
for i, var in enumerate(summary_X_train.var_name.values):
    
    if var in set(X_train_clean.columns):
        X_train_clean[var] = X_train_clean[var].astype(summary_X_train.data_type.values[i])
        X_test_clean[var]  = X_test_clean[var].astype(summary_X_train.data_type.values[i])

Pipeline consists of the following steps:
* Impute NA
* Transform categoricals
* Transform counts to log
* Normalize
* Derive polynomial terms
* Fit an elastic net model

## Transform Categoricals

Transform factors:

In [14]:
categ_bool = np.array(map(lambda x: x.name=='object', X_train_clean.dtypes.values) )
categ_cols = X_train_clean.columns[categ_bool==True]

In [15]:
dum_train = pd.get_dummies(X_train_clean[categ_cols], dummy_na=True, drop_first=True)

In [16]:
dum_test = pd.get_dummies(X_test_clean[categ_cols], dummy_na=True, drop_first=True)

In [17]:
check_columns(dum_train, dum_test)

{'left_only': {'month_6'}, 'right_only': set()}

In [18]:
# this is dirty
dum_train = dum_train.drop('month_6', axis=1)

In [19]:
X_train_clean = X_train_clean.drop(categ_cols, axis=1)
X_test_clean = X_test_clean.drop(categ_cols, axis=1)

## Impute + Normalize

In [20]:

from sklearn import preprocessing

In [21]:
imputer = preprocessing.Imputer()
scaler = preprocessing.StandardScaler()

In [22]:
imputer.fit(X_train_clean)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [23]:
arr_train = imputer.transform(X_train_clean)
arr_test  = imputer.transform(X_test_clean)

In [24]:
scaler.fit(arr_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [25]:
arr_train = scaler.transform(arr_train)
arr_test = scaler.transform(arr_test)

## Add back the dummies

In [26]:
X_train_clean = pd.DataFrame(arr_train, columns = X_train_clean.columns)

In [27]:
X_test_clean = pd.DataFrame(arr_test, columns = X_test_clean.columns)

In [28]:
X_train_clean = pd.concat([X_train_clean, dum_train], ignore_index=True, axis=1)
X_test_clean = pd.concat([X_test_clean, dum_test], ignore_index=True, axis=1)


In [32]:
y_train_clean = np.log(y_train.iloc[:,1]+1)

# Model comparison

ELASTIC NET REGRESSION

In [34]:
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import TimeSeriesSplit

In [35]:
tssplit = TimeSeriesSplit(n_splits=4)

In [36]:
enet = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=tssplit, n_jobs=-1)

In [37]:
enet.fit(X_train_clean, y_train_clean)

/home/roman/anaconda2/envs/python_cursus/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


ElasticNetCV(alphas=None, copy_X=True, cv=TimeSeriesSplit(n_splits=4),
       eps=0.001, fit_intercept=True,
       l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000,
       n_alphas=100, n_jobs=-1, normalize=False, positive=False,
       precompute='auto', random_state=None, selection='cyclic',
       tol=0.0001, verbose=0)

In [38]:
enet.alpha_

0.010905308476603847

In [39]:
enet.l1_ratio_

0.10000000000000001

In [40]:
predictions = enet.predict(X_test_clean)

In [41]:
transf_predictions = np.exp(predictions)-1

In [42]:
transf_predictions

array([ 5322429.52583446,  8673922.53073419,  5509406.13632862, ...,
        4559596.4620229 ,  4914232.51288776,  8232450.71103674])

In [43]:
df_test = pd.read_csv('~/Desktop/sberbank/test.csv')

df_sub = pd.DataFrame()
df_sub['id'] = df_test['id'].copy()
df_sub['price_doc'] = transf_predictions
df_sub.to_csv('./submission_enet_complete.csv')

RANDOM FOREST

In [44]:
from sklearn.ensemble import RandomForestRegressor

In [45]:
rf = RandomForestRegressor()

In [46]:
rf.fit(X_train_clean, y_train_clean)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [47]:
predictions = rf.predict(X_test_clean)
transf_predictions = np.exp(predictions)-1

In [48]:
transf_predictions

array([ 5253068.89548364,  8536746.94042377,  6725063.44099014, ...,
        4291980.3788267 ,  4953300.2112658 ,  8048476.99766481])

In [49]:
df_test = pd.read_csv('~/Desktop/sberbank/test.csv')

df_sub = pd.DataFrame()
df_sub['id'] = df_test['id'].copy()
df_sub['price_doc'] = transf_predictions
df_sub.to_csv('./submission_rf_complete.csv')

What else...

In [55]:
from sklearn.linear_model import LinearRegression

In [56]:
lr = LinearRegression()

In [57]:
lr.fit(X_train_clean, y_train_clean)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [58]:
predictions = lr.predict(X_test_clean)
transf_predictions = np.exp(predictions)-1

/home/roman/anaconda2/envs/python_cursus/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


In [60]:
transf_predictions

array([ 5218255.1613339 ,  8632198.61077119,  5820943.28423165, ...,
        3716632.54286415,  4118819.48605692,  6994103.85700917])